In [1]:
import pulp
import cplex

In [281]:

soc = [60,90,40,30,60]
n = len(soc)
stations = [20,30,50]
speed = 10
charging_rate_station = 10
ex_charging_rate_bess = 5
arrival_time = [1,3,4,7,8]
T = 1000
bess_capacity = 100

In [282]:
#initiate the model
cs = pulp.LpProblem("station",pulp.LpMinimize)
vehicles=[]
#add variables
soc_vehicle = {}
arrival_time_vehicle = {}
for i in range(n):
    vehicles.append(f"vehicle{i}")
    soc_vehicle[f"vehicle{i}"] = soc[i]
    arrival_time_vehicle[f"vehicle{i}"] = arrival_time[i]
M = 100000
print(vehicles)
#arrival time at stations
a1 = pulp.LpVariable.dict("Arrival Time 1",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
a2 = pulp.LpVariable.dict("Arrival Time 2",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
# servicing time at stations
s1 = pulp.LpVariable.dict("Service Time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
s2 = pulp.LpVariable.dict("Service Time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#initial charging time at sations
u1 = pulp.LpVariable.dict("initial charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
u2 = pulp.LpVariable.dict("initial charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#final charge time at stations
d1 = pulp.LpVariable.dict("final charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
d2 = pulp.LpVariable.dict("final charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# sigma- temporal ordering of vehicles
o1 = pulp.LpVariable.dict("sigma 1",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
o2 = pulp.LpVariable.dict("sigma 2",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
# to see if vehicle is gonna use the respective station
sb1 = pulp.LpVariable.dict("Service Time 1 binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
sb2 = pulp.LpVariable.dict("Service Time 2 binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
#bess servicing times at stations
b1 = pulp.LpVariable.dict("BESS service Time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
b2 = pulp.LpVariable.dict("BESS service Time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# charge in bess available for vehicles
bc1 = pulp.LpVariable.dict("BESS charge 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
bc2 = pulp.LpVariable.dict("BESS charge 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)





['vehicle0', 'vehicle1', 'vehicle2', 'vehicle3', 'vehicle4']


In [283]:
#objective function
cs += pulp.lpSum([d1[vehicle]-a1[vehicle] + d2[vehicle]-a2[vehicle]] for vehicle in vehicles)
#blending+= (pulp.lpSum([4.32*ing_weight[(i,"pork")]+2.46*ing_weight[(i,"wheat")]+1.86*ing_weight[(i,"starch")] for i in sausages]))

In [284]:
#constraints
for vehicle in vehicles:
    #departure = service + charge start
    cs += d1[vehicle] == s1[vehicle]+u1[vehicle]
    cs += d2[vehicle] == s2[vehicle]+u2[vehicle]
    
    # charge start > arival time
    cs += u1[vehicle]>=a1[vehicle]
    cs += u2[vehicle]>=a2[vehicle]
    cs += u2[vehicle]<=T-s2[vehicle]
    
    # arrival times of each station
    cs += a1[vehicle] == arrival_time_vehicle[vehicle]+(stations[0]/speed)
    cs += a2[vehicle] == (stations[1]/speed) + d1[vehicle]
    
    # ensuring soc capabilities
    cs += charging_rate_station*s1[vehicle]+ex_charging_rate_bess*b1[vehicle] + soc_vehicle[vehicle]>=stations[0]+stations[1]
    cs += charging_rate_station*s1[vehicle]+ex_charging_rate_bess*b1[vehicle] + soc_vehicle[vehicle]<=100
    cs += charging_rate_station*(s2[vehicle]+s1[vehicle])+ex_charging_rate_bess*(b2[vehicle]+b1[vehicle])+soc_vehicle[vehicle]>=stations[0]+stations[1]+stations[2]
    cs += charging_rate_station*(s2[vehicle]+s1[vehicle])+ex_charging_rate_bess*(b2[vehicle]+b1[vehicle])+soc_vehicle[vehicle]-stations[0]-stations[1]<=100
    
    #to see if a vehicle wants to cross the station or charge there
    cs += M*sb1[vehicle] >= s1[vehicle]
    cs += M*sb1[vehicle] >=s1[vehicle]
    cs += sb1[vehicle] <= s1[vehicle]
    cs += M*sb2[vehicle] >=s2[vehicle]
    cs += sb2[vehicle] <= s2[vehicle]
    # bess time should be less than total charging time at 
    cs += b1[vehicle]<=s1[vehicle]
    cs += b2[vehicle]<=s2[vehicle]
    cs+= b1[vehicle]>=bc1[vehicle]
    cs += b2[vehicle]>=bc2[vehicle]
    
# for bess implementations
for i in range(len(vehicles)):
    cs += bc1[vehicles[i]] == u1[vehicles[i]]-u1[vehicles[0]] - 2*pulp.lpSum([b1[vehicles[j]]] for j in range(i-1,-1,-1))
    cs += bc2[vehicles[i]] == u2[vehicles[i]]-u2[vehicles[0]] - 2*pulp.lpSum([b2[vehicles[j]]] for j in range(i-1,-1,-1))

# for waiting time implementation 
for vehicle1 in vehicles:
    
    for vehicle2 in vehicles:
        if vehicle1!=vehicle2:
            
            #making sure o1[vehicle1+vehicle2] = 1 when vehicle1 in front of vehicle2
            cs += M*o1[vehicle1+vehicle2] >= a1[vehicle2]-a1[vehicle1]
            cs += M*(1-o1[vehicle1+vehicle2])>= a1[vehicle1]-a1[vehicle2]
            cs += M*o2[vehicle1+vehicle2] >= a2[vehicle2]-a2[vehicle1]
            cs += M*(1-o2[vehicle1+vehicle2]) >= a2[vehicle1]-a2[vehicle2]
            
            #implementing wait time
            cs += u1[vehicle2]-u1[vehicle1]-s1[vehicle1]>=(o1[vehicle1+vehicle2]-1)*T-M*(1-sb1[vehicle2])
            cs += u2[vehicle2]-u2[vehicle1]-s2[vehicle1]>=(o2[vehicle1+vehicle2]-1)*T-M*(1-sb2[vehicle2])
            
            #making sure only one vehicle is front of another
            cs += o1[vehicle1+vehicle2] + o1[vehicle2+vehicle1] >= 1
            cs += o2[vehicle1+vehicle2] + o2[vehicle2+vehicle1] >= 1




In [285]:
cs

station:
MINIMIZE
-1*Arrival_Time_1_vehicle0 + -1*Arrival_Time_1_vehicle1 + -1*Arrival_Time_1_vehicle2 + -1*Arrival_Time_1_vehicle3 + -1*Arrival_Time_1_vehicle4 + -1*Arrival_Time_2_vehicle0 + -1*Arrival_Time_2_vehicle1 + -1*Arrival_Time_2_vehicle2 + -1*Arrival_Time_2_vehicle3 + -1*Arrival_Time_2_vehicle4 + 1*final_charge_time_1_vehicle0 + 1*final_charge_time_1_vehicle1 + 1*final_charge_time_1_vehicle2 + 1*final_charge_time_1_vehicle3 + 1*final_charge_time_1_vehicle4 + 1*final_charge_time_2_vehicle0 + 1*final_charge_time_2_vehicle1 + 1*final_charge_time_2_vehicle2 + 1*final_charge_time_2_vehicle3 + 1*final_charge_time_2_vehicle4 + 0
SUBJECT TO
_C1: - Service_Time_1_vehicle0 + final_charge_time_1_vehicle0
 - initial_charge_time_1_vehicle0 = 0

_C2: - Service_Time_2_vehicle0 + final_charge_time_2_vehicle0
 - initial_charge_time_2_vehicle0 = 0

_C3: - Arrival_Time_1_vehicle0 + initial_charge_time_1_vehicle0 >= 0

_C4: - Arrival_Time_2_vehicle0 + initial_charge_time_2_vehicle0 >= 0

_C5: Se

In [286]:
cs.solve(CPLEX_PY())

, best possible 62.5 (3255.94 seconds)
Cbc0010I After 8640000 nodes, 97519 on tree, 68 best solution, best possible 62.5 (3256.27 seconds)
Cbc0010I After 8641000 nodes, 97343 on tree, 68 best solution, best possible 62.5 (3256.56 seconds)
Cbc0010I After 8642000 nodes, 97476 on tree, 68 best solution, best possible 62.5 (3256.89 seconds)
Cbc0010I After 8643000 nodes, 97438 on tree, 68 best solution, best possible 62.5 (3257.18 seconds)
Cbc0010I After 8644000 nodes, 97395 on tree, 68 best solution, best possible 62.5 (3257.51 seconds)
Cbc0010I After 8645000 nodes, 97054 on tree, 68 best solution, best possible 62.5 (3257.77 seconds)
Cbc0010I After 8646000 nodes, 97146 on tree, 68 best solution, best possible 62.5 (3258.10 seconds)
Cbc0010I After 8647000 nodes, 97050 on tree, 68 best solution, best possible 62.5 (3258.43 seconds)
Cbc0010I After 8648000 nodes, 96984 on tree, 68 best solution, best possible 62.5 (3258.77 seconds)
Cbc0010I After 8649000 nodes, 96632 on tree, 68 best solution

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sushi/Desktop/internship/Routing/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d1232812f63346398dc10867282d957c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/d1232812f63346398dc10867282d957c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 275 COLUMNS
At line 1299 RHS
At line 1570 BOUNDS
At line 1681 ENDATA
Problem MODEL has 270 rows, 110 columns and 781 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 16.5 - 0.00 seconds
Cgl0003I 2 fixed, 15 tightened bounds, 19 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 8 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 2 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0004I processed model has 137 rows, 61 columns (61 integer (28 of which binary)) and 427 el

1

In [287]:
pulp.LpStatus[cs.status]

'Optimal'

In [290]:
for vehicle in vehicles:
    print(f"{vehicle} waits at s1 for {u1[vehicle].varValue-a1[vehicle].varValue}, s2: for {u2[vehicle].varValue-a2[vehicle].varValue}")
    print(f"{vehicle} charges at s1 for {b1[vehicle].varValue} using the bess and station, and for {s1[vehicle].varValue-b1[vehicle].varValue} only using station")
    print(f"{vehicle} charges at s2 for {b2[vehicle].varValue} using the bess and station, and for {s2[vehicle].varValue-b2[vehicle].varValue} only using station")
    print(f"{vehicle} reaches s1 , s2 at {a1[vehicle].varValue, a2[vehicle].varValue}")
    print()

vehicle0 waits at s1 for 0.0, s2: for 0.0
vehicle0 charges at s1 for 0.0 using the bess and station, and for 0.0 only using station
vehicle0 charges at s2 for 1.0 using the bess and station, and for 3.0 only using station
vehicle0 reaches s1 , s2 at (3.0, 6.0)

vehicle1 waits at s1 for 0.0, s2: for 0.0
vehicle1 charges at s1 for 0.0 using the bess and station, and for 1.0 only using station
vehicle1 charges at s2 for 0.0 using the bess and station, and for 0.0 only using station
vehicle1 reaches s1 , s2 at (5.0, 9.0)

vehicle2 waits at s1 for 0.0, s2: for 0.0
vehicle2 charges at s1 for 3.0 using the bess and station, and for 0.0 only using station
vehicle2 charges at s2 for 1.0 using the bess and station, and for 0.0 only using station
vehicle2 reaches s1 , s2 at (6.0, 12.0)

vehicle3 waits at s1 for 0.0, s2: for 0.0
vehicle3 charges at s1 for 1.0 using the bess and station, and for 1.0 only using station
vehicle3 charges at s2 for 3.0 using the bess and station, and for 0.0 only using


Cbc0010I After 9236000 nodes, 50994 on tree, 68 best solution, best possible 63.898256 (3425.97 seconds)
Cbc0010I After 9237000 nodes, 50953 on tree, 68 best solution, best possible 63.898256 (3426.24 seconds)
Cbc0010I After 9238000 nodes, 50874 on tree, 68 best solution, best possible 63.906431 (3426.62 seconds)
Cbc0010I After 9239000 nodes, 50769 on tree, 68 best solution, best possible 63.906431 (3426.92 seconds)
Cbc0010I After 9240000 nodes, 50714 on tree, 68 best solution, best possible 63.906431 (3427.19 seconds)
Cbc0010I After 9241000 nodes, 50690 on tree, 68 best solution, best possible 63.906431 (3427.44 seconds)
Cbc0010I After 9242000 nodes, 50548 on tree, 68 best solution, best possible 63.916667 (3427.73 seconds)
Cbc0010I After 9243000 nodes, 50442 on tree, 68 best solution, best possible 63.916667 (3427.97 seconds)
Cbc0010I After 9244000 nodes, 50347 on tree, 68 best solution, best possible 63.916667 (3428.25 seconds)
Cbc0010I After 9245000 nodes, 50311 on tree, 68 best s

In [289]:
print(pulp.value(cs.objective))

18.0


s, 95485 on tree, 68 best solution, best possible 62.505022 (3268.66 seconds)
Cbc0010I After 8681000 nodes, 95481 on tree, 68 best solution, best possible 62.505022 (3268.98 seconds)
Cbc0010I After 8682000 nodes, 95524 on tree, 68 best solution, best possible 62.522222 (3269.42 seconds)
Cbc0010I After 8683000 nodes, 95472 on tree, 68 best solution, best possible 62.522222 (3269.74 seconds)
Cbc0010I After 8684000 nodes, 95444 on tree, 68 best solution, best possible 62.522222 (3270.07 seconds)
Cbc0010I After 8685000 nodes, 95330 on tree, 68 best solution, best possible 62.522222 (3270.32 seconds)
Cbc0010I After 8686000 nodes, 95489 on tree, 68 best solution, best possible 62.545455 (3270.65 seconds)
Cbc0010I After 8687000 nodes, 95434 on tree, 68 best solution, best possible 62.545455 (3270.94 seconds)
Cbc0010I After 8688000 nodes, 95385 on tree, 68 best solution, best possible 62.545455 (3271.29 seconds)
Cbc0010I After 8689000 nodes, 95058 on tree, 68 best solution, best possible 62.54